In [326]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [327]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [328]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Mirosław,Kaźmierkiewicz,62053019019,Wrocław,ul. Wiosenna 95/8,48011819213,miroslaw.kazmierkiewicz@kuba.pl,2014-01-01,None,menadżer,72.7
1,2,Văn Tân,Musiałowski,60101120917,Wrocław,ul. Kościuszki 85/50,48254632945,van tan.musialowski@kuba.pl,2014-01-01,None,sprzedawca,45.2
2,3,Łukasz,Wrażeń,73030785616,Lębork,ul. Ignacego Daszyńskiego 7,48947065097,lukasz.wrazen@kuba.pl,2014-01-01,None,sprzedawca,46.4
3,4,Marek,Kędzierski,75101487610,Radom,ul. Wspólna 37/15,48318840125,marek.kedzierski@kuba.pl,2014-01-01,None,mechanik,56.2
12,5,Grażyna,Jeruszka,80100568785,Wrocław,ul. Józefa Chełmońskiego 5/34,48784303015,grazyna.jeruszka@kuba.pl,2014-01-01,None,informatyk,58.4
4,6,Jerzy,Galeano,87091073799,Wrocław,ul. Wiosenna 74/50,48529519180,jerzy.galeano@kuba.pl,2016-02-03,2017-07-29,mechanik,58.6
5,7,Tytus,Lekki,86111248414,Wrocław,ul. Łagiewnicka 94/42,48657437391,tytus.lekki@kuba.pl,2016-02-23,None,mechanik,53.3
6,8,Rafał,Ignasiak,92111051912,Wrocław,ul. Morelowa 39/11,48179202440,rafal.ignasiak@kuba.pl,2016-03-01,None,mechanik,56.1
7,9,Bartosz,Nowak,75120654936,Wrocław,ul. Sąsiedzka 83/4,48730531393,bartosz.nowak@kuba.pl,2016-09-20,None,pomocnik mechanika,38.7
8,10,Dominika,Marszałek,79041959860,Wrocław,ul. Truskawkowa 62/44,48773223243,dominika.marszalek@kuba.pl,2016-10-02,None,pomocnik mechanika,35.4


In [329]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [330]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [331]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [332]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4]) for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [333]:
len(year_ceil), len(pd.unique(client_ids))

(1579, 1579)

In [334]:
year_ceil

[2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [335]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [336]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [337]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [338]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [339]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [340]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Halina,Włośkiewicz,K,62071538839,Wrocław,ul. Lipowa 60/45,False,48675372220,z0x485zl3d@opayq.com
1,2,Jolanta,Mika,K,85062660452,Gorzów Wielkopolski,ul. Księcia Przemysława 87/1,False,48367420461,j.mika@gmail.com
2,3,Adrian,Turek,M,03260769658,Wrocław,ul. Wiosłowa 87/10,False,48196533240,adrture@gmail.com
3,4,Beata,Woźniak,K,72090622617,Wrocław,ul. Poli Gojawiczyńskiej 97/18,False,48931485699,wozniabeata72@onet.pl
4,5,Krzysztof,Woźnica,M,86100755556,Wrocław,ul. Szkolna 39/8,True,48653958965,k-woznica1986@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1574,1575,Adam,Próchniak,M,None,Wąchock,ul. Sosnowa 79/42,False,48610840106,pro_ada1996@wp.pl
1575,1576,Tomasz,Nawara,M,82051464833,Wrocław,ul. Różana 54/20,False,48732342960,t.nawa@gmail.com
1576,1577,Anna,Kondraciuk,K,90041854252,Wrocław,ul. Rzemieślnicza 28/41,False,48268448715,a-kon@gmail.com
1577,1578,Katarzyna,Domańska,K,73042872852,Wąbrzeźno,ul. Borowina 22/50,False,48770635993,domanska732@gmail.com


In [341]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [342]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Halina,Włośkiewicz,K,62071538839,Wrocław,ul. Lipowa 60/45,False,48675372220,z0x485zl3d@opayq.com
1,2,Jolanta,Mika,K,85062660452,Gorzów Wielkopolski,ul. Księcia Przemysława 87/1,False,48367420461,j.mika@gmail.com
2,3,Adrian,Turek,M,03260769658,Wrocław,ul. Wiosłowa 87/10,False,48196533240,adrture@gmail.com
3,4,Beata,Woźniak,K,72090622617,Wrocław,ul. Poli Gojawiczyńskiej 97/18,False,48931485699,wozniabeata72@onet.pl
4,5,Krzysztof,Woźnica,M,86100755556,Wrocław,ul. Szkolna 39/8,True,48653958965,k-woznica1986@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1574,1575,Adam,Próchniak,M,None,Wąchock,ul. Sosnowa 79/42,False,48610840106,pro_ada1996@wp.pl
1575,1576,Tomasz,Nawara,M,82051464833,Wrocław,ul. Różana 54/20,False,48732342960,t.nawa@gmail.com
1576,1577,Anna,Kondraciuk,K,90041854252,Wrocław,ul. Rzemieślnicza 28/41,False,48268448715,a-kon@gmail.com
1577,1578,Katarzyna,Domańska,K,73042872852,Wąbrzeźno,ul. Borowina 22/50,False,48770635993,domanska732@gmail.com


In [343]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [344]:
np.sum(clients_df['karta_lojalnościowa'])

80

In [345]:
np.sum(pd.isna(clients_df["PESEL"]))

67

# auta

In [346]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [347]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [348]:
df_samochody = create_car_table(max(car_ids))

In [349]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,SEAT,Mii,4,2002,Kombi,False,srebrny,Benzyna,0.9,60,Manualna,5,5,False
1,2,Dacia,Sandero,4,2010,Kombi,True,brązowy,Benzyna,1.4,90,Manualna,3,5,False
2,3,Fiat,Panda,4,2009,Sedan,False,szary,Benzyna,1.1,86,Automatyczna,5,2,False
3,4,BMW,440,4,2000,Van,False,szary,Benzyna,5.0,326,Automatyczna,5,5,False
4,5,Volkswagen,Touareg,4,2006,Sedan,False,szary,Diesel,3.7,231,Automatyczna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,1014,BMW,525,4,2008,Hatchback,False,czarny,Diesel,3.5,231,Automatyczna,3,5,False
1014,1015,Ford,Grand Tourneo,4,2011,SUV,False,szary,Diesel,2.0,116,Manualna,5,5,False
1015,1016,Ford,Mondeo,4,2009,Kombi,True,srebrny,Diesel,2.5,163,Manualna,3,5,False
1016,1017,Opel,Adam,4,2005,Hatchback,False,biały,Benzyna,1.5,87,Manualna,5,5,True


In [350]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [351]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna', 'Półautomatyczna'], dtype=object)

# Usługi

In [352]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [353]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [354]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [355]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [356]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [357]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,17800.0
1,2,2,2,3,2014-01-02,26900.0
2,3,3,3,3,2014-01-03,40300.0
3,4,4,4,2,2014-01-04,95000.0
4,5,5,5,3,2014-01-04,62700.0
...,...,...,...,...,...,...
978,979,946,1489,2,2017-12-27,39100.0
979,980,1016,845,3,2017-12-28,45800.0
980,981,443,1213,2,2017-12-28,36600.0
981,982,1017,637,2,2017-12-29,25300.0


In [358]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [359]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [360]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():
        sell_prices[i] *= 0.97

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\1097819302.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():


[12, 12, 18, 6, 6, 6, 1, 18, 1, 12, 12, 6, 6, 1, 18, 1, 6, 6, 18, 18, 1, 12, 12, 6, 18, 1, 1, 18, 6, 18, 6, 6, 1, 6, 18, 6, 1, 6, 6, 6, 1, 6, 1, 18, 18, 1, 6, 6, 1, 6, 6, 12, 1, 12, 1, 6, 12, 1, 6, 1, 1, 6, 6, 12, 1, 1, 18, 12, 12, 6, 1, 1, 18, 6, 1, 6, 1, 6, 1, 1, 18, 6, 18, 12, 18, 1, 18, 1, 1, 18, 12, 1, 18, 12, 12, 1, 6, 6, 12, 1, 6, 18, 12, 18, 1, 12, 1, 12, 12, 1, 1, 1, 18, 1, 6, 18, 1, 1, 1, 12, 6, 6, 6, 18, 12, 6, 6, 6, 1, 1, 1, 1, 12, 1, 1, 6, 12, 6, 6, 12, 6, 6, 18, 1, 6, 12, 1, 6, 18, 6, 1, 12, 6, 1, 6, 6, 1, 6, 6, 1, 6, 12, 18, 1, 1, 1, 18, 6, 6, 6, 1, 1, 12, 1, 12, 1, 12, 1, 18, 12, 18, 1, 12, 12, 18, 18, 18, 1, 1, 1, 1, 12, 1, 6, 12, 1, 1, 1, 6, 1, 1, 1, 6, 18, 1, 1, 1, 6, 18, 1, 12, 12, 6, 18, 12, 18, 12, 12, 6, 1, 1, 1, 12, 12, 1, 1, 18, 12, 1, 1, 6, 6, 12, 18, 1, 6, 6, 6, 1, 1, 1, 12, 12, 18, 1, 18, 12, 1, 18, 1, 18, 12, 18, 18, 6, 6, 12, 6, 18, 12, 18, 12, 12, 1, 1, 18, 1, 6, 1, 6, 6, 1, 6, 1, 12, 1, 6, 1, 6, 6, 18, 12, 18, 1, 18, 18, 18, 18, 6, 1, 18, 12, 1, 6, 1, 12

In [361]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,17,21,2,40144.0,12,2014-01-12
1,2,18,22,3,49816.0,12,2014-01-12
2,3,6,23,3,71020.0,18,2014-01-12
3,4,2,24,2,40290.0,6,2014-01-14
4,5,16,25,3,44064.0,6,2014-01-14
...,...,...,...,...,...,...,...
975,976,946,1577,3,63700.0,1,2017-12-28
976,977,443,1461,2,66606.0,6,2017-12-30
977,978,1006,50,3,30804.0,6,2017-12-31
978,979,1018,1059,3,65500.0,1,2017-12-31


In [362]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [363]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [364]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(25, array([4], dtype=int64)), (36, array([30], dtype=int64)), (42, array([25], dtype=int64)), (44, array([4], dtype=int64)), (52, array([56], dtype=int64)), (57, array([38], dtype=int64)), (60, array([26], dtype=int64)), (63, array([28], dtype=int64)), (70, array([46, 33], dtype=int64)), (88, array([27, 41], dtype=int64)), (108, array([27], dtype=int64)), (117, array([63, 14], dtype=int64)), (133, array([8], dtype=int64)), (143, array([38], dtype=int64)), (146, array([27, 39], dtype=int64)), (156, array([8], dtype=int64)), (198, array([ 0, 10], dtype=int64)), (203, array([26], dtype=int64)), (212, array([55], dtype=int64)), (219, array([40], dtype=int64)), (222, array([34, 46], dtype=int64)), (242, array([40, 22], dtype=int64)), (257, array([32, 48,  3], dtype=int64)), (262, array([60], dtype=int64)), (263, array([47], dtype=int64)), (265, array([22, 46], dtype=int64)), (277, array([26], dtype=int64)), (287, array([41], dtype=int64)), (295, array([45], dtype=int64)), (298, array([23]

In [365]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [366]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Felga,R15 czarny,143,1,4
1,Dywan,BMW 114,14,2,2
2,Światła tylne,Peugeot 208,49,3,1
3,Felga,R16 czerwony,18,4,4
4,Kierownica,Volkswagen Polo,41,5,1
...,...,...,...,...,...
191,Fotel przedni,Peugeot 108 skóra,169,133,1
192,Drzwi lewe przednie,Mercedes-Benz E 200 czerwony,686,134,1
193,Akumulator,smart forTwo,303,135,1
194,Klocek hamulcowy,smart forTwo,11,135,4


# Wyposażenie

In [367]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [368]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Felga,R15 czarny,143.0
1,2,Dywan,BMW 114,14.0
2,3,Światła tylne,Peugeot 208,49.0
3,4,Felga,R16 czerwony,18.0
4,5,Kierownica,Volkswagen Polo,41.0
...,...,...,...,...
191,145,Fotel przedni,Peugeot 108 skóra,169.0
192,146,Drzwi lewe przednie,Mercedes-Benz E 200 czerwony,686.0
193,147,Akumulator,smart forTwo,303.0
194,148,Klocek hamulcowy,smart forTwo,11.0


# Usługi - uzupełnienie

In [369]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [370]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,25,49,6,2014-01-30,760
1,2,36,70,4,2014-02-16,40
2,3,42,81,6,2014-02-25,70
3,4,44,85,6,2014-02-27,100
4,5,52,99,4,2014-03-09,50
...,...,...,...,...,...,...
131,132,973,1524,8,2017-10-27,460
132,133,980,1534,4,2017-11-08,240
133,134,985,315,6,2017-11-10,910
134,135,1001,1559,6,2017-12-06,460


In [371]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [372]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [373]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,4
1,2,2,2,2
2,3,3,3,1
3,4,4,4,4
4,5,5,5,1
...,...,...,...,...
191,192,133,145,1
192,193,134,146,1
193,194,135,147,1
194,195,135,148,4


In [374]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [375]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [376]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,850.0,7.0
1,2.0,2.0,75.0,6.0
2,3.0,3.0,170.0,4.0
3,4.0,4.0,104.0,7.0
4,5.0,5.0,132.0,4.0
...,...,...,...,...
144,145.0,145.0,633.0,4.0
145,146.0,146.0,1160.0,2.0
146,147.0,147.0,504.0,2.0
147,148.0,148.0,89.0,10.0


In [377]:
parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [378]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [379]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)